In [2]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [3]:
from sklearn.model_selection import train_test_split

#픽셀 평준화
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2)

# **심층 신경망**

In [8]:
#시그모이드 함수 사용층
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),name='hidden')
#소프트맥스 함수 사용층
dense2 = keras.layers.Dense(10, activation='softmax',name='output')
#입력층 -> dense1(은닉층) -> dense2(출력층)
model = keras.Sequential([dense1, dense2])
model.summary()
#Layer(type):레이어 이름과 타입
#Output Shape:출력 배열의 형태. None은 나중에 지정 가능하다는 의미이다.
#Param:각 층의 가중치와 절편의 총 개수
#은닉층(hidden)의 경우 입력층의 출력이 784개이고 100개의 뉴런이 있으므로 784*100(가중치 개수)+100(각 뉴런마다 한개의 절편)으로 Param값은 78500
#출력층(output)의 경우 은닉층의 출력이 100개이고 10개의 뉴런이 있으므로 100*10(가중치 개수)+10(각 뉴런마다 한개의 절편)으로 Param값은 1010

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**층을 추가하는 다른 방법**

In [9]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
#권장
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax',name='output'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 2ms/step - loss: 0.7738 - accuracy: 0.7538
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4187 - accuracy: 0.8512
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3739 - accuracy: 0.8654
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3458 - accuracy: 0.8752
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3348 - accuracy: 0.8809


**렐루 활성함수와 Flatten층**

In [15]:
model = keras.Sequential()
#Flatten층:편의를 위한 층으로 입력으로 받은 다차원 배열을 1차원 배열로 펼쳐준다.
model.add(keras.layers.Flatten(input_shape=(28, 28)))
#렐루(relu) 활성함수는 시그모이드함수보다 학습을 더 빠르고 효율적으로 할 수 있게 도와준다.
model.add(keras.layers.Dense(100, activation='relu',name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax',name='output'))
model.summary()
#아래를 보면 Flatten층은 파라미터가 0개이고 출력으로 784(28*28)개의 1차원 배열임을 알 수 있다.

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [17]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2,)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6841 - accuracy: 0.7638
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4074 - accuracy: 0.8551
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3569 - accuracy: 0.8728
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3408 - accuracy: 0.8780
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3295 - accuracy: 0.8824


In [18]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3539 - accuracy: 0.8788


[0.3539038896560669, 0.8788333535194397]

# **옵티마이저**
*optimizer매개변수는 모델의 최적화 방법을 설정할 수 있다.(기본값:'sgd')

In [19]:
#모델 초기화 함수
def init_model():
  model = keras.Sequential()  
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(100, activation='relu',name='hidden'))
  model.add(keras.layers.Dense(10, activation='softmax',name='output'))

In [23]:
#sgd:확률적 경사 하강법('확률적경사하강법' 참고)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')
#learning_rate:학습률(기본값:0.01)
sgd = keras.optimizers.SGD(learning_rate=0.1)
#momentum:모멘텀 방식 사용률(기본값:0)
#nesterov:네스테로프 모멘텀 방식 사용 여부(기본값:False)
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

**적응형 학습률 옵티마이저**

In [20]:
init_model()
#Adagrad
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')
init_model()
#RMSprop
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [21]:
init_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3055 - accuracy: 0.8903
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2887 - accuracy: 0.8939
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2667 - accuracy: 0.9004
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2594 - accuracy: 0.9027
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2432 - accuracy: 0.9087


In [22]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3215 - accuracy: 0.8854


[0.32152169942855835, 0.8854166865348816]

# **앙상블 모델과 인공신경망 모델의 차이점**

앙상블 모델과 인공신경망 모델은 여러개의 트리모델/뉴런을 사용하여 출력값을 낸다는 점에서 비슷하다고 볼 수 있다. 하지만 앙상블 모델은 여러개의 트리모델이 독립적으로 학습되는 데 반해, 인공신경망 모델은 모든 뉴런(밀집층의 경우)이 같이 학습되며, 앙상블모델과는 다르게 여러개의 층을 쌓을 수 있다는 장점이 있다.
 이러한 차이점 때문에 앙상블 모델은 DB나 엑셀과 같은 정형화된 데이터 학습에 효과적이며, 인공신경망 모델은 이미지등의 비정형화된 데이터 학습에 효과적이다.